<a href="https://colab.research.google.com/github/bajayo/docs/blob/master/test_tools_and_message_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Install openai package - specify the version here. For the first test use "1.7.1", for the second "1.9.0".
!pip install openai==1.24.0
!pip install nest_asyncio

# Import required libraries
import asyncio
from openai import AsyncOpenAI
from openai import AsyncAssistantEventHandler
from openai.types.beta.threads.runs import RunStep, ToolCall
from typing_extensions import override
import nest_asyncio

# create an assystant with the following setup:
# instructions:
# You are a python 3.11 Full Stack Developer assistant. You are a fastAPI, Cloud, Google Cloud, expert.
# The user interacts with you while building and testing a Business Cloud infrastructure layer that delivers advanced AI capabilities to the organization's employees.
# Your job is to support the development team that works on that infrastructure when testing, troubleshooting, debugging and creating new features and applications on the platform.
# The user may ask to simulate various responses for mimicking various scenarios. You should follow the user's instructions and generate exactly the output requested.
# The developer may ask you to return text, files, function calls or any combination of them.


# model: gpt-4-turbo-preview

# enable File search
# enable Code interpreter


# add the following function
ask_question_function = {
  "name": "ask_question",
  "description": "This function should always be triggered for asking the user any question.",
  "parameters": {
    "type": "object",
    "properties": {
      "title": {
        "type": "string",
        "description": "The title of the question that should be presented to the user."
      },
      "text": {
        "type": "string",
        "description": "The text of the question that should be presented to the user."
      },
      "id": {
        "type": "string",
        "description": "An identifier representing the question. For example 'age'."
      },
      "tooltip": {
        "type": "string",
        "description": "A text describing the question and its purpose."
      },
      "summary": {
        "type": "string",
        "description": "A textual summary of the question and answers that will be used for providing the context to the next call to the ai engine. For example: The assistant asked to provide age range, the question id is 'age', the possible answers are: 'age_1': '18-40', 'age_2': '41+'."
      },
      "type": {
        "type": "string",
        "description": "The type of the question. The options are: 'single_choice': the user can only choose one answer, 'multi_choice': the user may select more than one answer"
      },
      "answers": {
        "type": "object",
        "description": "A list of all the possible answer objects.",
        "properties": {
          "answer_text": {
            "type": "string",
            "description": "The text that the user should see."
          },
          "answer_id": {
            "type": "string",
            "description": "An id representing the answer in the following structure: {question_id}_{number}"
          },
          "answer_tooltip": {
            "type": "string",
            "description": "A text describing the answer."
          }
        },
        "required": [
          "answer_text",
          "answer_id"
        ]
      }
    },
    "required": [
      "title",
      "text",
      "id",
      "summary",
      "type",
      "answers"
    ]
  }
}


# Import required libraries
import asyncio
from openai import AsyncOpenAI

# Your API_KEY and ASSISTANT_ID go here
API_KEY = "your_api_key_here"
ASSISTANT_ID = "your_assistant_id_here" # use the one you created

# Apply the patch to allow nested event loops
nest_asyncio.apply()

# The main coroutine for asynchronous execution
async def main():
    # Initialize the AsyncOpenAI client
    client = AsyncOpenAI(
        api_key=API_KEY
    )

    content = "Please return a text response: \"this is a text response\" and a sample call the ask_question function, ask for my age. Please return both responses to this message. Start with the text response and follow with the tool_call (ask_question) response."
    ##############################################################################
    # 1 - No streaming
    # Create a thread
    print("Run without streaming")
    thread = await client.beta.threads.create()
    print(f"thread id = {thread.id}")
    # Add a message to the thread
    message = await client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=content
    )
    print(f"message id = {message.id}")

    # Run the assistant
    run = await client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=ASSISTANT_ID
    )
    print(f"run id = {run.id}")

    # Poll run status
    while run.status not in ["completed", "requires_action", "cancelled", "failed", "expired"]:
        print(f"run status= {run.status}")
        run = await client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
        await asyncio.sleep(1)  # wait 1 sec
    print(f"run final status= {run.status}")
    # when status is "requires_action"
    if run.status == "requires_action":
        print("required_action = ", run.required_action.json())

    # When run status is "completed" or "requires_action", print the message output.
    messages = await client.beta.threads.messages.list(
                thread_id=thread.id
            )
    print("messages = ", messages.json())

    ##############################################################################
    # 2 - with streaming:
    print("Run with streaming")
    thread = await client.beta.threads.create()
    print(f"thread id = {thread.id}")

    # Add a message to the thread
    message = await client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=content
    )
    print(f"message id = {message.id}")

    event_handler = EventHandler()

    async with client.beta.threads.runs.stream(
          thread_id=thread.id,
          assistant_id=ASSISTANT_ID,
          event_handler=event_handler
    ) as stream:
      await stream.until_done()



class EventHandler(AsyncAssistantEventHandler):

    @override
    async def on_text_created(self, text) -> None:
        print(f"on_text_created: {text.value}")

    @override
    async def on_text_done(self, text) -> None:
        print(f"on_text_done: text = {text.value}")

    @override
    async def on_run_step_created(self, run_step: RunStep):
        print(f"on_run_step_created: {run_step.json()}")

    @override
    async def on_run_step_done(self, run_step: RunStep):
        print(f"on_run_step_done: run step = {run_step.json()}")


    @override
    async def on_tool_call_created(self, tool_call: ToolCall):
        print(f"on_tool_call_created: tool call = {tool_call.json()}")

    @override
    async def on_tool_call_done(self, tool_call):
        print(f"on_tool_call_done: tool call = {tool_call.json()}")

    @override
    async def on_end(self):
        print("on_end")



print("Starting to run")

# Get the current loop and run the main coroutine
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

Starting to run
Run without streaming
thread id = thread_6GvJ89JCRo3qf4uMPR4gDJKg
message id = msg_3yuyi9OPPhX6ICD81cwevn4I
run id = run_ZwjehDD5Q1btPK9EIEHwppQz
run status= queued
run status= in_progress
run status= in_progress
run status= in_progress
run status= in_progress
run status= in_progress
run status= in_progress
run status= in_progress
run final status= requires_action
required_action =  {"submit_tool_outputs":{"tool_calls":[{"id":"call_r5EdghbIhbSR8rP5Onb7bR79","function":{"arguments":"{\"title\":\"What is your age?\",\"text\":\"Please select the age range that applies to you. This information helps us tailor our responses to better suit your needs.\",\"id\":\"age\",\"summary\":\"The assistant asked the user for their age range to tailor responses better.\",\"type\":\"single_choice\",\"answers\":[{\"answer_text\":\"Under 18\",\"answer_id\":\"age_1\"},{\"answer_text\":\"18-24\",\"answer_id\":\"age_2\"},{\"answer_text\":\"25-34\",\"answer_id\":\"age_3\"},{\"answer_text\":\"35